In [ ]:
import sagemaker
bucket = sagemaker.session.Session().default_bucket()

## Amazon Polly

### synthesize_speech

In [ ]:
import boto3
polly = boto3.client('polly')
response = polly.synthesize_speech(
    # 作成する音声の文
    Text = '''
    はじめて Amazon Polly で音声を作成しました。
    ''',
    OutputFormat = "mp3", # 出力形式を選択
    VoiceId = 'Takumi', # 声を選択
)
# 音声データをファイルに出力する
with open('./first_voice.mp3', "wb") as f:
    f.write(response['AudioStream'].read())


### SSML

In [ ]:
response = polly.synthesize_speech(
    Text = '''
    <speak>
        <amazon:breath duration="medium" volume="x-loud"/>
　　　　私は <break time="1s"/>
　　　　<emphasis>SSML</emphasis>
　　　　で音声を作成しました
    </speak>
    ''',
    OutputFormat = "mp3", 
    TextType='ssml', # SSML であることを明示する
    VoiceId = 'Takumi', # 声を選択
)
with open('./ssml_voice.mp3', "wb") as f:
    f.write(response['AudioStream'].read())

### Lexicon

In [ ]:
LEXICON_NAME = 'emacs'
CONTENT = '''<?xml version="1.0" encoding="UTF-8"?>
<lexicon version="1.0"
      xmlns="http://www.w3.org/2005/01/pronunciation-lexicon"
      xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xsi:schemaLocation="http://www.w3.org/2005/01/pronunciation-lexicon
        http://www.w3.org/TR/2007/CR-pronunciation-lexicon-20071212/pls.xsd"
      alphabet="ipa"
      xml:lang="ja-JP">
  <lexeme><grapheme>テキストエディタ</grapheme><phoneme>ˈiːˌmæks</phoneme></lexeme>
</lexicon>'''
response = polly.put_lexicon(Content=CONTENT,Name=LEXICON_NAME)

In [ ]:
# polly.delete_lexicon(Name=LEXICON_NAME)

In [ ]:
response = polly.synthesize_speech(
    Text = 'テキストエディタを開きましょう',
    OutputFormat = "mp3",
    VoiceId = 'Takumi',
    LexiconNames=[LEXICON_NAME]
)
with open('./emacs.mp3', "wb") as f:
    f.write(response['AudioStream'].read())

response = polly.synthesize_speech(
    Text = 'テキストエディタを開きましょう',
    OutputFormat = "mp3",
    VoiceId = 'Takumi',
)
with open('./text_editor.mp3', "wb") as f:
    f.write(response['AudioStream'].read())

## Amazon Transcribe

### start_transcription_job

In [ ]:
s3 = boto3.client('s3') # 事前に音声ファイルを S3 にアップロード
AUDIO_FILE='emacs.mp3'
AUDIO_KEY = f'transcribe/input/{AUDIO_FILE}'
s3.upload_file(f'./{AUDIO_FILE}', bucket, AUDIO_KEY)
transcribe = boto3.client('transcribe')
JOBNAME = 'my_first_transcription_job'
OUTPUT_KEY='transcribe/output/'
response = transcribe.start_transcription_job(
    TranscriptionJobName=JOBNAME,
    LanguageCode='ja-JP',
    Media={
        'MediaFileUri': f's3://{bucket}/{AUDIO_KEY}'
    },
    OutputBucketName=bucket,
    OutputKey=OUTPUT_KEY,
)

In [ ]:
# transcribe = boto3.client('transcribe')
# JOBNAME = 'my_first_transcription_job'
# transcribe.delete_transcription_job(TranscriptionJobName=JOBNAME)

In [ ]:
transcript_result_uri = transcribe.get_transcription_job(TranscriptionJobName=JOBNAME)['TranscriptionJob']['Transcript']['TranscriptFileUri']
boto3.client('s3').download_file(bucket, f'{OUTPUT_KEY}{JOBNAME}.json', f'./{JOBNAME}.json')
with open(f'./{JOBNAME}.json','r') as f:print(f.read())

### create_vocabulary

In [ ]:
CUSTOM_VOCABULARY = 'Phrase\tIPA\tSoundsLike\tDisplayAs\nemacs\t\tiimakkusu\tEmacs'
CUSTOM_VOCABULARY_CSV = './custom_vocabulary.txt'
with open(CUSTOM_VOCABULARY_CSV,'w') as f:f.write(CUSTOM_VOCABULARY)
print(CUSTOM_VOCABULARY)

In [ ]:
VOCABULARY_NAME = 'emacs'
VOCABULARY_KEY = f'transcribe/custom_vocaburary/{CUSTOM_VOCABULARY_CSV}'
s3.upload_file(f'./{CUSTOM_VOCABULARY_CSV}', bucket, VOCABULARY_KEY)
transcribe.create_vocabulary(
    VocabularyName=VOCABULARY_NAME,
    LanguageCode='ja-JP',
    VocabularyFileUri=f's3://{bucket}/{VOCABULARY_KEY}',
)

In [ ]:
# transcribe.delete_vocabulary(VocabularyName=VOCABULARY_NAME)

In [ ]:
transcribe = boto3.client('transcribe')
JOBNAME = 'custom_vocabulary_transcription_job'
transcribe.delete_transcription_job(TranscriptionJobName=JOBNAME)

In [ ]:
JOBNAME = 'custom_vocabulary_transcription_job'
response = transcribe.start_transcription_job(
    TranscriptionJobName=JOBNAME,
    LanguageCode='ja-JP',
    Settings={'VocabularyName': VOCABULARY_NAME},
    Media={
        'MediaFileUri': f's3://{bucket}/{AUDIO_KEY}'
    },
    OutputBucketName=bucket,
    OutputKey=OUTPUT_KEY,
)

In [ ]:
transcript_result_uri = transcribe.get_transcription_job(TranscriptionJobName=JOBNAME)['TranscriptionJob']['Transcript']['TranscriptFileUri']
boto3.client('s3').download_file(bucket, f'{OUTPUT_KEY}{JOBNAME}.json', f'./{JOBNAME}.json')
with open(f'./{JOBNAME}.json','r') as f:print(f.read())

## Amazon Translate

### translate_text

In [ ]:
translate = boto3.client('translate')
response = translate.translate_text(
    # 翻訳させたい文字列
    Text='I use white vaseline daily to protect my skin.',
    # 翻訳元の言語
    SourceLanguageCode='en',
    # 翻訳したい言語
    TargetLanguageCode='ja',
    Settings={
        'Profanity': 'MASK'
    }
)
print(response['TranslatedText'])


### import_terminologies

In [ ]:
TERMINOLOGIES = 'en,ja\nwhite vaseline,白色ワセリン'.encode()
TERMINOLOGIES_NAME = 'white_vaseline'
translate.import_terminology(
    Name=TERMINOLOGIES_NAME,
    MergeStrategy='OVERWRITE',
    TerminologyData={
        'File': TERMINOLOGIES,
        'Format': 'CSV',
    }
)
response = translate.translate_text(
    Text='I use white vaseline daily to protect my skin.',
    # terminology の指定
    TerminologyNames=[TERMINOLOGIES_NAME],
    SourceLanguageCode='en',
    TargetLanguageCode='ja',
)
print(response['TranslatedText'])

## Amazon Comprehend

### detect_dominant_language

In [ ]:
comprehend = boto3.client('comprehend')
TEXT = 'Toutes les grandes personnes ont d’abord été des enfants, mais peu d’entre elles s’en souviennent.'
response = translate.translate_text(
    Text=TEXT,
    SourceLanguageCode=comprehend.detect_dominant_language(Text=TEXT)['Languages'][0]['LanguageCode'],
    TargetLanguageCode='ja',
)
print(response['TranslatedText'])

### detect_entities